# Particle Trapping with Muon decay

Notes:
- Simulation is a three stage deal:
    - Generate muons isotropically within the DS trap region, allow them to propigate for 700 ns (no decay)
    - Use that output to further evolve remaining muons, allowing them to decay to electrons.
    - Run full output through TrkAna, see if any tracks are reconstructed.
- Their are two copies of each 2nd stage muon in `ntpart`.
    - Looks like copy 1 has t==0, p==100, pstop=p at 700ns
    - The second muon has t=~700, p==100, parent_pstop = original p_stop
- No muons have an initial timestamp > 700 ns
- No electrons are born before 700 ns


### Some Findings

Number of trapped muons per init muon p:
* p=140: 33000*300 = ~9.9mil

Number of unique events in which an electron with p>100 makes it farther upstream than 11200mm:
* Muon init p=140: ? Events

In [ ]:
from mu2e import mu2e_ext_path
from mu2e.dataframeprod import g4root_to_df
from root_pandas import read_root
import pandas as pd
from mu2e.mu2eplots import mu2e_plot3d_ptrap
from mu2e.mu2eplots import mu2e_plot3d_ptrap_traj
from mu2e.mu2eplots import mu2e_plot3d_ptrap_anim
import cPickle as pkl
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [3]:
with open('/Volumes/DataDump/Mu2E/jobs_stage2.txt', 'r') as job_file:
    print job_file.readline()

15613699



In [2]:
import mmap
def get_line_number(file_path):  
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

# Bookkeeping for high-stats samples

In [3]:
from root_pandas import to_root, read_root
import tqdm
jobs_list_tmp = [15613699, 15613700, 15613702, 15613704]
jobs_path = '/Volumes/DataDump/Mu2E/jobs_stage2.txt'
#for i in tqdm.tqdm_notebook(jobs_list_tmp):
with open(jobs_path, 'r') as job_file:
    for line in tqdm.tqdm_notebook(job_file, total=get_line_number(jobs_path)):
        i = int(line)
        input_root = '/Volumes/DataDump/Mu2E/stage2-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_stage2_vd.v0.all.root'.format(i)
        output_root = '/Volumes/DataDump/Mu2E/stage2-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_stage2_vd.v0.skim.root'.format(i)
        df_ntpart = read_root(input_root, 'readvd/ntpart', ignore='*vd')
        df_nttvd = read_root(input_root, 'readvd/nttvd')
        #df_ntvd = read_root(input_root, 'readvd/ntvd')
    
        df_ntpart.set_index(['evt', 'subrun'], inplace=True)
        
        df_nttvd.set_index(['evt', 'subrun'], inplace=True)
        #df_ntvd.set_index(['evt', 'subrun'], inplace=True)
    
        good_index = df_ntpart.query('pdg==11 and p>75').index
        df_ntpart = df_ntpart.ix[good_index]
        #df_ntvd = df_ntvd.ix[good_index]
        df_nttvd = df_nttvd.ix[good_index]
        #df_ntpart['job'] = i
        df_ntpart.eval('job = {}'.format(i), inplace=True)
        df_ntpart.eval('x = x+3904', inplace=True)
        df_ntpart.eval('xstop = xstop+3904', inplace=True)
        #df_ntvd['job'] = i
        #df_nttvd['job'] = i
        df_nttvd.eval('job = {}'.format(i), inplace=True)
        df_nttvd.eval('x = x+3904', inplace=True)
        df_nttvd.eval('p = sqrt(px**2+py**2+pz**2)', inplace=True)
    
        df_ntpart.reset_index(inplace=True)
        df_nttvd.reset_index(inplace=True)
        #df_ntvd.reset_index(inplace=True)
    
        #df_ntvd.to_root(output_root, key='ntvd', mode = 'w')
        df_nttvd.to_root(output_root, key='nttvd', mode = 'w')
        df_ntpart.to_root(output_root, key='ntpart', mode = 'a')
    #print df_nttvd.head()


In [12]:
input_root = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/15669486/00/00000/nts.bpollack.iso_muons_GA05_trkana.v0.all.root'
df_trkana = read_root(input_root, 'TrkAna/trkana', ['evtinfo', 'dem', 'uem', 'dmm'])

In [11]:
df_trkana

,evtinfo_eventid,evtinfo_runid,evtinfo_subrunid,evtinfo_evtwt,evtinfo_beamwt,evtinfo_genwt,evtinfo_nprotons,dem_status,dem_pdg,dem_nhits,...,dem_d0,dem_p0,dem_om,dem_z0,dem_td,dem_d0err,dem_p0err,dem_omerr,dem_z0err,dem_tderr
0,124270,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,699995,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,841510,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1088000,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1100909,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1240922,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1396909,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1493249,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1510464,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1532568,1,0,1.0,1.0,1.0,-1,-1000,93,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
import re
from root_pandas import to_root, read_root
import tqdm
jobs_path = '/Volumes/DataDump/Mu2E/jobs_trkana.txt'
with open(jobs_path, 'r') as job_file:
    for line in tqdm.tqdm_notebook(job_file, total=get_line_number(jobs_path)):
        i = int(line)
        input_root = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_trkana.v0.all.root'.format(i)
        input_log = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/{}/00/00000/log.bpollack.trkana-140p-hack.v0.all.log'.format(i)
        output_root = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/{}/00/00000/nts.bpollack.iso_muons_GA05_trkana.v0.skim.root'.format(i)
        
        df_trkana = read_root(input_root, 'TrkAna/trkana', ['evtinfo', 'dem', 'uem', 'dmm'])
        
        with open(input_log) as log_file:
            m = re.search('(?<=stage2-140p-hack/outstage/)\w+', log_file.read())
        jn = int(m.group(0))
        
        df_trkana['evtinfo_job'] = jn
        
        df_trkana.to_root(output_root, key='trkana', mode = 'w')
        

In [13]:
input_log = '/Volumes/DataDump/Mu2E/trkana-140p-hack/outstage/15669486/00/00000/log.bpollack.trkana-140p-hack.v0.all.log'

In [37]:
import re
with open(input_log) as f:
    m = re.search('(?<=stage2-140p-hack/outstage/)\w+', f.read())
jn = m.group(0)

In [41]:
print int(jn)

In [27]:
m.group(0)

'stage2'

In [42]:
print 'hello'